# EXTREME GRADIENT REGRESSOR ALGORITHEM

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv('DATASET.csv')
df.head()

,Age,Credit_Score,Account_Balence,Credit_Card_Holter,Dependents,Employment_Type,Applicant_Income,Coapplicant_Income,Loan_Status,Education_Status,Fidelity_Scores
0,86,451,825749,1,1,Employed,128630,81915,0,Graduated,51
1,64,465,140667,0,0,Employed,75633,78479,1,Graduated,59
2,65,409,716824,0,2,Self_Employed,178202,174249,0,Non-Graduated,99
3,57,446,323539,0,2,Employed,180802,96209,1,Non-Graduated,28
4,67,475,973045,1,2,Employed,80110,25057,1,Non-Graduated,23


In [3]:
df.columns

Index(['Age', 'Credit_Score', 'Account_Balence', 'Credit_Card_Holter',
       'Dependents', 'Employment_Type', 'Applicant_Income',
       'Coapplicant_Income', 'Loan_Status', 'Education_Status',
       'Fidelity_Scores'],
      dtype='object')

In [4]:
df=df.dropna()

In [5]:
df.head()

,Age,Credit_Score,Account_Balence,Credit_Card_Holter,Dependents,Employment_Type,Applicant_Income,Coapplicant_Income,Loan_Status,Education_Status,Fidelity_Scores
0,86,451,825749,1,1,Employed,128630,81915,0,Graduated,51
1,64,465,140667,0,0,Employed,75633,78479,1,Graduated,59
2,65,409,716824,0,2,Self_Employed,178202,174249,0,Non-Graduated,99
3,57,446,323539,0,2,Employed,180802,96209,1,Non-Graduated,28
4,67,475,973045,1,2,Employed,80110,25057,1,Non-Graduated,23


In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

var = ['Employment_Type','Education_Status']
  
for i in var:
    df[i] = le.fit_transform(df[i]).astype(int)

In [7]:
x = df.drop(labels='Fidelity_Scores', axis=1)
y = df.loc[:,'Fidelity_Scores']

In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
print("NUMBER OF TRAIN DATASET    : ", len(x_train))
print("NUMBER OF TEST DATASET      : ", len(x_test))
print("TOTAL NUMBER OF DATASET    : ", len(x_train)+len(x_test))

NUMBER OF TRAIN DATASET    :  16000
NUMBER OF TEST DATASET      :  4000
TOTAL NUMBER OF DATASET    :  20000


In [9]:
print("NUMBER OF TRAIN DATASET    : ", len(y_train))
print("NUMBER OF TEST DATASET      : ", len(y_test))
print("TOTAL NUMBER OF DATASET    : ", len(y_train)+len(y_test))

NUMBER OF TRAIN DATASET    :  16000
NUMBER OF TEST DATASET      :  4000
TOTAL NUMBER OF DATASET    :  20000


In [10]:
from xgboost import XGBRegressor

In [11]:
xgb = XGBRegressor()
xgb.fit(x_train,y_train)

[10:40:11] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor()

In [12]:
predicted = xgb.predict(x_test)

In [13]:
from sklearn.metrics import rand_score
RS = rand_score(y_test, predicted)
print("MEAN RAND SCOORE OF EXTREME GRADIENT REGRESSOR ALGORITHEM",RS*100)

MEAN RAND SCOORE OF EXTREME GRADIENT REGRESSOR ALGORITHEM 98.7627281820455


In [14]:
from sklearn.metrics import r2_score
R2 = r2_score(y_test, predicted)
print("MEAN R2 SCOORE OF EXTREME GRADIENT REGRESSOR ALGORITHEM",R2*100)

MEAN R2 SCOORE OF EXTREME GRADIENT REGRESSOR ALGORITHEM -0.5059112283036127


In [15]:
from sklearn.metrics import mean_squared_error
MSE = mean_squared_error(y_test, predicted)
print("MEAN SQUARE ERROR OF EXTREME GRADIENT REGRESSOR ALGORITHEM",MSE)

MEAN SQUARE ERROR OF EXTREME GRADIENT REGRESSOR ALGORITHEM 541.8181286771597


In [16]:
from sklearn.metrics import mean_squared_error
RMSE = mean_squared_error(y_test, predicted, squared=False)
print(" ROOT MEAN SQUARE ERROR OF EXTREME GRADIENT REGRESSOR ALGORITHEM",RMSE)

 ROOT MEAN SQUARE ERROR OF EXTREME GRADIENT REGRESSOR ALGORITHEM 23.276987104802885


In [17]:
from sklearn.metrics import mean_absolute_error
MAE = mean_absolute_error(y_test, predicted)
print("MEAN ABSOLUTE ERROR OF EXTREME GRADIENT REGRESSOR ALGORITHEM",MAE)

MEAN ABSOLUTE ERROR OF EXTREME GRADIENT REGRESSOR ALGORITHEM 20.075546250343322


In [18]:
from sklearn.metrics import explained_variance_score
EVS = explained_variance_score(y_test, predicted)
print("EXPLAINED VARIANCE SCORE OF EXTREME GRADIENT REGRESSOR ALGORITHEM",EVS)

EXPLAINED VARIANCE SCORE OF EXTREME GRADIENT REGRESSOR ALGORITHEM -0.00400829581174067


In [20]:
import joblib
joblib.dump(xgb, 'XGB.pkl')

['XGB.pkl']